# Main file for regression models (exlcuding LSTM)

In [ ]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')
from regression_helper_functions_03 import ts_data, train_test, train_validation, plot_acutal_predict, model_performance, rmse, convert, performance_table, naive_baseline, grid_search_ses, grid_search_gbr, gbr, grid_search_rf, grid_search_sarima, sarima, arima_predict, grid_search_arima, performance_table_single
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima_model import ARIMA
import timeit

In [ ]:
def run_naive_baseline(CONT, FREQ, PREDICT_HORIZON):
    MODEL_RESULTS = dict()
    for i in CONT:
        MODEL_RESULTS[i] = {'MODEL':[],'RUN_TIME':[], 'Frequency':[], 'Prediction_Window':[], 'RMSE':[], 'NRMSE':[], 'MAPE':[], 'ND':[]}
    
    for c in CONT:
            for f in FREQ:
                for p in PREDICT_HORIZON:
                    x = ts_data(country = c, category = 'total', frequency = f, model = None)

                    #augmented out of sample training 
                    train_s = int(len(x)*0.7)
                    test_s = len(x) - train_s
                    t = list()
                    t_ = list()
                    idx = list()

                    START_TIME = timeit.default_timer()

                    for i in range(0,int(test_s/p)):
                        train, test = train_test(x,train_s,p)

                        y_hat = naive_baseline(train, test)
                        idx = idx + list(test.index)
                        t_ = t_ + list(y_hat.values)
                        t = t + list(test.values)
                        train_s += p

                    END_TIME = timeit.default_timer()
                    TIME = convert(END_TIME-START_TIME)

                    y_real = pd.DataFrame(t,index=idx)
                    y_pred = pd.DataFrame(t_,index=idx)
                    mape, rmse, nrmse, nd = model_performance(y_real, y_pred)

                    MODEL_RESULTS[c]['MODEL'].append('NAIVE_BASELINE')
                    MODEL_RESULTS[c]['RUN_TIME'].append(TIME)
                    MODEL_RESULTS[c]['Frequency'].append(f)
                    MODEL_RESULTS[c]['Prediction_Window'].append(p)
                    MODEL_RESULTS[c]['RMSE'].append(rmse)
                    MODEL_RESULTS[c]['MAPE'].append(mape)
                    MODEL_RESULTS[c]['NRMSE'].append(nrmse)
                    MODEL_RESULTS[c]['ND'].append(nd)
            
    return performance_table(MODEL_RESULTS)

In [ ]:
#Simple Exponential Smoothing
def run_ses(CONT, FREQ, PREDICT_HORIZON):
    
    MODEL_RESULTS = dict()
    for i in CONT:
        MODEL_RESULTS[i] = {'MODEL':[],'RUN_TIME':[], 'Frequency':[], 'Prediction_Window':[], 'RMSE':[], 'NRMSE':[], 'MAPE':[], 'ND':[]}
    
    for c in CONT:
        for f in FREQ:
            for p in PREDICT_HORIZON:
                x = ts_data(country = c, category = 'total', frequency = f, model = None)
                #grid search using 70% of the original data
                train_data_gs, test_data_gs = train_test(x,int(len(x)*0.7),0)
                alpha = grid_search_ses(train_data_gs)

                #augmented out of sample training 
                train_s = int(len(x)*0.7)
                test_s = len(x) - train_s
                t = list()
                t_ = list()
                idx = list()
                
                START_TIME = timeit.default_timer()
                
                for i in range(0,int(test_s/p)):
                    train, test = train_test(x,train_s,p)

                    y_hat = SimpleExpSmoothing(train).fit(smoothing_level=alpha,optimized=False).forecast(p)

                    idx = idx + list(test.index)
                    t_ = t_ + list(y_hat.values)
                    t = t + list(test.values)
                    train_s += p
                    
                END_TIME = timeit.default_timer()
                TIME = convert(END_TIME-START_TIME)
                
                y_real = pd.DataFrame(t,index=idx)
                y_pred = pd.DataFrame(t_,index=idx)
                mape, rmse, nrmse, nd = model_performance(y_real, y_pred)
                
                MODEL_RESULTS[c]['MODEL'].append('SES')
                MODEL_RESULTS[c]['RUN_TIME'].append(TIME)
                MODEL_RESULTS[c]['Frequency'].append(f)
                MODEL_RESULTS[c]['Prediction_Window'].append(p)
                MODEL_RESULTS[c]['RMSE'].append(rmse)
                MODEL_RESULTS[c]['MAPE'].append(mape)
                MODEL_RESULTS[c]['NRMSE'].append(nrmse)
                MODEL_RESULTS[c]['ND'].append(nd)
            
    return performance_table(MODEL_RESULTS)

In [ ]:
#GBR
def run_gbr(CONT, FREQ, PREDICT_HORIZON):
    
    MODEL_RESULTS = dict()
    for i in CONT:
        MODEL_RESULTS[i] = {'MODEL':[],'RUN_TIME':[], 'Frequency':[], 'Prediction_Window':[], 'RMSE':[], 'NRMSE':[], 'MAPE':[], 'ND':[]}
    
    for c in CONT:
        for f in FREQ:
            for p in PREDICT_HORIZON:
                x, dt = ts_data(country = c, category = 'total', frequency = f, model = 'gbr_rf')
                #grid search using 70% of the original data
                train_data_gs, test_data_gs = train_test(x,int(len(x)*0.7),0)
                gbr_parameters = grid_search_gbr(train_data_gs)

                #augmented out of sample training 
                train_s = int(len(x)*0.7)
                test_s = len(x) - train_s
                t = list()
                t_ = list()
                idx = list()
                
                START_TIME = timeit.default_timer()
                
                for i in range(0,int(test_s/p)):
                    train, test = train_test(x,train_s,p)

                    train_input = train.drop(['y'], axis=1)
                    test_input = test.drop(['y'], axis=1)

                    y_hat = gbr(train_input,train['y'],test_input,gbr_parameters)

                    idx = idx + list(test.index)
                    t_ = t_ + list(y_hat)
                    t = t + list(test['y'].values)
                    train_s += p
                    
                END_TIME = timeit.default_timer()
                TIME = convert(END_TIME-START_TIME)
                
                y_real = pd.DataFrame(t,index=idx)
                y_pred = pd.DataFrame(t_,index=idx)
                mape, rmse, nrmse, nd = model_performance(y_real, y_pred)

                MODEL_RESULTS[c]['MODEL'].append('GBR')
                MODEL_RESULTS[c]['RUN_TIME'].append(TIME)
                MODEL_RESULTS[c]['Frequency'].append(f)
                MODEL_RESULTS[c]['Prediction_Window'].append(p)
                MODEL_RESULTS[c]['RMSE'].append(rmse)
                MODEL_RESULTS[c]['MAPE'].append(mape)
                MODEL_RESULTS[c]['NRMSE'].append(nrmse)
                MODEL_RESULTS[c]['ND'].append(nd)
                
    return performance_table(MODEL_RESULTS)

In [ ]:
#RF
def run_rf(CONT, FREQ, PREDICT_HORIZON):
    MODEL_RESULTS = dict()
    for i in CONT:
        MODEL_RESULTS[i] = {'MODEL':[],'RUN_TIME':[], 'Frequency':[], 'Prediction_Window':[], 'RMSE':[], 'NRMSE':[], 'MAPE':[], 'ND':[]}
    for c in CONT:
        for f in FREQ:
            for p in PREDICT_HORIZON:
                x, dt = ts_data(country = c, category = 'total', frequency = f, model = 'gbr_rf')
                #grid search using 70% of the original data
                train_data_gs, test_data_gs = train_test(x,int(len(x)*0.7),0)
                rf_parameters = grid_search_rf(train_data_gs)

                #augmented out of sample training 
                train_s = int(len(x)*0.7)
                test_s = len(x) - train_s
                t = list()
                t_ = list()
                idx = list()

                START_TIME = timeit.default_timer()

                for i in range(0,int(test_s/p)):
                    train, test = train_test(x,train_s,p)

                    train_input = train.drop(['y'], axis=1)
                    test_input = test.drop(['y'], axis=1)

                    model = RandomForestRegressor(n_estimators = rf_parameters[0], max_features = rf_parameters[1], 
                                                  max_depth = rf_parameters[2], random_state = 42).fit(train_input, train['y'])
                    y_hat = model.predict(test_input)

                    idx = idx + list(test.index)
                    t_ = t_ + list(y_hat)
                    t = t + list(test['y'].values)
                    train_s += p

                END_TIME = timeit.default_timer()
                TIME = convert(END_TIME-START_TIME)

                y_real = pd.DataFrame(t,index=idx)
                y_pred = pd.DataFrame(t_,index=idx)
                mape, rmse, nrmse, nd = model_performance(y_real, y_pred)

                MODEL_RESULTS[c]['MODEL'].append('RF')
                MODEL_RESULTS[c]['RUN_TIME'].append(TIME)
                MODEL_RESULTS[c]['Frequency'].append(f)
                MODEL_RESULTS[c]['Prediction_Window'].append(p)
                MODEL_RESULTS[c]['RMSE'].append(rmse)
                MODEL_RESULTS[c]['MAPE'].append(mape)
                MODEL_RESULTS[c]['NRMSE'].append(nrmse)
                MODEL_RESULTS[c]['ND'].append(nd)

    return performance_table(MODEL_RESULTS)

In [ ]:
#ARIMA
def run_arima(CONT, FREQ, PREDICT_HORIZON):
    
    MODEL_RESULTS = dict()
    for i in CONT:
        MODEL_RESULTS[i] = {'MODEL':[],'RUN_TIME':[], 'Frequency':[], 'Prediction_Window':[], 'RMSE':[], 'NRMSE':[], 'MAPE':[], 'ND':[]}
    
    for c in CONT:
        for f in FREQ:
            for p in PREDICT_HORIZON:
                x = ts_data(country = c, category = 'total', frequency = f, model = None)
                #grid search using 70% of the original data
                train_data_gs, test_data_gs = train_test(x,int(len(x)*0.7),0)
                arima_parameters = grid_search_arima(train_data_gs)

                #augmented out of sample training 
                train_s = int(len(x)*0.7)
                test_s = len(x) - train_s
                t = list()
                t_ = list()
                idx = list()
                
                START_TIME = timeit.default_timer()
                
                for i in range(0,int(test_s/p)):
                    train, test = train_test(x,train_s,p)
                    y_hat = ARIMA(train, arima_parameters).fit(disp=False).predict(start=len(train),end=len(train)+p-1)

                    idx = idx + list(test.index)
                    t_ = t_ + list(y_hat.values)
                    t = t + list(test.values)
                    train_s += p
                    
                END_TIME = timeit.default_timer()
                TIME = convert(END_TIME-START_TIME)
                
                y_real = pd.DataFrame(t,index=idx)
                y_pred = pd.DataFrame(t_,index=idx)
                mape, rmse, nrmse, nd = model_performance(y_real, y_pred)              
                
                MODEL_RESULTS[c]['MODEL'].append('ARIMA')
                MODEL_RESULTS[c]['RUN_TIME'].append(TIME)
                MODEL_RESULTS[c]['Frequency'].append(f)
                MODEL_RESULTS[c]['Prediction_Window'].append(p)
                MODEL_RESULTS[c]['RMSE'].append(rmse)
                MODEL_RESULTS[c]['MAPE'].append(mape)
                MODEL_RESULTS[c]['NRMSE'].append(nrmse)
                MODEL_RESULTS[c]['ND'].append(nd)

    return performance_table(MODEL_RESULTS)

In [ ]:
def run_sarima(CONT, FREQ, PREDICT_HORIZON):
    MODEL_RESULTS = dict()
    for i in CONT:
        MODEL_RESULTS[i] = {'MODEL':[],'RUN_TIME':[], 'Frequency':[], 'Prediction_Window':[], 'RMSE':[], 'NRMSE':[], 'MAPE':[], 'ND':[]}
    
    for c in CONT:
        for f in FREQ:
            for p in PREDICT_HORIZON:
                x = ts_data(country = c, category = 'total', frequency = f, model = None)
                #grid search using 70% of the original data
                train_data_gs, test_data_gs = train_test(x,int(len(x)*0.7),0)
                pqd = grid_search_sarima(train_data_gs)

                #augmented out of sample training 
                train_s = int(len(x)*0.7)
                test_s = len(x) - train_s
                t = list()
                t_ = list()
                idx = list()
                
                START_TIME = timeit.default_timer()
                
                for i in range(0,int(test_s/p)):

                    train, test = train_test(x,train_s,p)
                    y_hat = sarima(pqd[0][0],pqd[0][1],pqd[0][2],pqd[1][0],pqd[1][1],pqd[1][2],train
                                  ).get_forecast(steps=p).conf_int()
                    y_hat['y'] = (y_hat['lower y'] + y_hat['upper y'])/2
        
                    idx = idx + list(test.index)
                    t_ = t_ + list(y_hat['y'].values)
                    t = t + list(test.values)

                    train_s += p
                    
                    
                END_TIME = timeit.default_timer()
                TIME = convert(END_TIME-START_TIME)
                
                y_real = pd.DataFrame(t,index=idx)
                y_pred = pd.DataFrame(t_,index=idx)
                mape, rmse, nrmse, nd = model_performance(y_real, y_pred)
                
                print('c, f, p', c, f, p, 'parameters', pqd)
                
                MODEL_RESULTS[c]['MODEL'].append('SARIMA')
                MODEL_RESULTS[c]['RUN_TIME'].append(TIME)
                MODEL_RESULTS[c]['Frequency'].append(f)
                MODEL_RESULTS[c]['Prediction_Window'].append(p)
                MODEL_RESULTS[c]['RMSE'].append(rmse)
                MODEL_RESULTS[c]['MAPE'].append(mape)
                MODEL_RESULTS[c]['NRMSE'].append(nrmse)
                MODEL_RESULTS[c]['ND'].append(nd)
                print(MODEL_RESULTS)
    return performance_table_single(MODEL_RESULTS)

In [ ]:
FREQ = ['2H']
PREDICT_HORIZON = [6,12,24]
CONT = ['CA']

TABLE_SARIMA = run_sarima(CONT, FREQ, PREDICT_HORIZON)

TABLE_SARIMA.to_csv('Regression_Performance_SARIMA_CA_P2.csv', index = False)

In [ ]:
%%capture
FREQ = ['1H','2H']
PREDICT_HORIZON = [1, 6, 12, 24, 72]
CONT = ['US','CA','GB','ENTIRE','NORTH_AMERICA']

TABLE_NAIVE_BASELINE = run_naive_baseline(CONT, FREQ, PREDICT_HORIZON)
TABLE_SES = run_ses(CONT, FREQ, PREDICT_HORIZON)
TABLE_GBR = run_gbr(CONT, FREQ, PREDICT_HORIZON)
TABLE_RF = run_rf(CONT, FREQ, PREDICT_HORIZON)
TABLE_ARIMA = run_arima(CONT, FREQ, PREDICT_HORIZON)

#save output to csv
TABLES = [TABLE_SES, TABLE_GBR, TABLE_NAIVE_BASELINE, TABLE_RF, TABLE_ARIMA]
REGRESSION_PERFORMANCE = pd.concat(TABLES)

REGRESSION_PERFORMANCE.to_csv('Regression_Performance.csv', index = False)

In [ ]:
%%capture
FREQ = ['24H']
PREDICT_HORIZON = [1, 6, 12, 24]
CONT = ['US','CA','GB','ENTIRE','NORTH_AMERICA']

TABLE_SES = run_ses(CONT, FREQ, PREDICT_HORIZON)
TABLE_GBR = run_gbr(CONT, FREQ, PREDICT_HORIZON)
TABLE_RF = run_rf(CONT, FREQ, PREDICT_HORIZON)
TABLE_ARIMA = run_arima(CONT, FREQ, PREDICT_HORIZON)

#save output to csv
TABLES = [TABLE_SES, TABLE_GBR, TABLE_RF, TABLE_ARIMA]
REGRESSION_PERFORMANCE = pd.concat(TABLES)

REGRESSION_PERFORMANCE.to_csv('Regression_Performance_24H.csv', index = False)